In [0]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
'''info'''
#Recall that the most optimal series found yet was 
#series = 600/(1.88686*ii+1)
#with an overshot of 0.004491383366257651 km using 6 stops.

#series = 600/(2.054*ii+1)
#Base was reached with an overshot of 0.09309210393803369 km using 7 stops. - This one works 
#The rover made it! The configuration works. The rover made it to the base with 0.15789206767922348 of fuel left over.
#THIS ONE DOESN'T WORK BECAUSE THE ROVER PICKS UP TOO MUCH FUEL. LOOK AT FIXING? 

#REMEMBER TO ADD A SECTION THAT TRACKS THE TOTAL AMOUNT OF FUEL USED.
#DOES NOT SEEM TO WORK FOR THE MORE 'EFFICIENT' SERIES. THINK ABOUT WHY THAT MAY BE.

#Build an expression/graph that looks at how close different harmonic series come to reaching 2. Newton-Rhapson? Graphical?

#Is the minimum number of stops 7?

#Harmonic numbers

#No actual fuel used in travelling back - due to using the stops. Extra fuel use is compensated for by having to use a finer sequence. 

#Define the forwards journey code and the forwards and backwards journey code as functions so that they can easily be iterated over multiple 
#different values for graphs of fuel used against distance to be used etc. 

'info'

In [3]:
'''Testing different series to see which one converges the best'''
stops = 0
summ = 0

for ii in range(1,100):
  series = 600/(2*ii)
  summ += series
  stops = ii
  print(series)
  if summ >= 1200:
    print('Base was reached with an overshot of', summ-1200, 'km using', stops, 'stops.')
    break 

300.0
150.0
100.0
75.0
60.0
50.0
42.857142857142854
37.5
33.333333333333336
30.0
27.272727272727273
25.0
23.076923076923077
21.428571428571427
20.0
18.75
17.647058823529413
16.666666666666668
15.789473684210526
15.0
14.285714285714286
13.636363636363637
13.043478260869565
12.5
12.0
11.538461538461538
11.11111111111111
10.714285714285714
10.344827586206897
10.0
9.67741935483871
Base was reached with an overshot of 8.17355863095645 km using 31 stops.


In [4]:
'''Start converting the problem into code. Begin by intialising values to be used later'''
total_distance = 2 #Have it as 2 for the moment to measure everything in 'tank lots' of the landrover
fuel_used = 0 #Keep track of the total amount of fuel used
tank_level = 1 #to keep track of the fuel in the tank at any moment
lower_tank_threshold = -5e-15 #The lowest we will allow the tank level to reach while still allowing for the effects of numerical estimation.

#2d array to contain information about the stops. 
#column one details the distance the outpost is at from the origin(constant) 
#while column two details displacement of the outpost from the previous outpost
#and column three details the fuel deposit at the outpost
stop_info = np.zeros((int(stops), 3))
summ=0

for ii in range(stops, 0, -1):
  series = 1/(2*ii)
  stop_info[ii-1,1]=series
  summ += series
  stop_info[ii-1,0] = summ

#Reversing the order of the array 
stop_info=stop_info[::-1]




'''Begin simulation of the actual journey using the series'''
#Include an if statement within the for loop that will immediately catch if the tank of the rover falls below 0
failed_statement='This configuration does not work. The tank has reached negative fuel and the rover is stranded.'

#Introduce a nested for set of for loops. One that will deal with the establishment of new outposts, and the other that will reflect the fuel used from previously 
#established ones to get to the new outpost 
for current_stop in range(0,stops-1):
  print('Establishing stop', current_stop)
  
  for ii in range(0, current_stop):
    #Forwards direction
    tank_level=tank_level-stop_info[ii,1] #Displacement only from the last stop, not the total distance from the origin.
    if tank_level<lower_tank_threshold:
      print(failed_statement, stop_info)
      break 
    stop_info[ii,2]=stop_info[ii,2]-stop_info[ii,1] #Removing fuel from each passed stop to account for the pickup of fuel to travel forward
    tank_level=tank_level + stop_info[ii,1] #Gaining the fuel from the stop
    if tank_level<lower_tank_threshold:
      print(failed_statement, stop_info)
      break 
  
  #Travelling past established stops and setting up new stop
  tank_level=tank_level - stop_info[current_stop,1] #forwards journey only
  fuel_used+=2*stop_info[current_stop,1] #Accounting for fuel for forwards and backwards journey
  if tank_level<lower_tank_threshold:
    print(failed_statement, stop_info)
    break 
  stop_info[current_stop, 2] = tank_level - stop_info[current_stop,1] #Depositing fuel while leaving enough to reach the next stop
  tank_level = tank_level - stop_info[current_stop, 2] #Reflecting the fuel lost to establish the outpost 
  fuel_used+=stop_info[current_stop,2] #Fuel to set up the outpost
  if tank_level<lower_tank_threshold:
    print(failed_statement, stop_info)
    break 
  for jj in range(current_stop, 0, -1): 
    #Backwards direction
    tank_level=tank_level-stop_info[jj,1] #Removing fuel to reach the next outpost
    if tank_level<lower_tank_threshold:
      print(failed_statement, stop_info)
      break 
    stop_info[jj-1, 2] = stop_info[jj-1,2]-stop_info[jj-1,1] #Removing one unit of fuel from the outpost to refuel the tank
    tank_level = tank_level+stop_info[jj-1,1]
    if tank_level<lower_tank_threshold:
      print(failed_statement, stop_info)
      break 

  #account for the last stop
  tank_level=tank_level-stop_info[0,0] 
  if tank_level<lower_tank_threshold:
    print(failed_statement, stop_info)
    break 

  #Refuel at reaching the landing zone
  print('Reached LZ with a tank of', tank_level)
  tank_level=1
  print(stop_info)

print('Prepared for final journey. Fuel used has been', fuel_used)




'''Now include a section that actually carries out the final journey to ensure the series used has been viable'''
print('\n \n \n Beginning the final journey. Fuel is at', tank_level)
for final_journey in range(0, stops-1):
    tank_level=tank_level - stop_info[final_journey,1]
    print('\n Reached outpost', final_journey, 'with', tank_level, 'of fuel.')
    tank_level = tank_level + 0.5*stop_info[final_journey,2]
    if tank_level>1.1:
        print('Tank reached', tank_level)
        break
    stop_info[final_journey,2] = stop_info[final_journey,2]-stop_info[final_journey,1]
    print('The stops are at \n', stop_info)

final_leg = 2-stop_info[-2,0] #The distance after the final stop
print('\n The journey left is', final_leg , 'and the rover has', tank_level, 'of fuel')
tank_level=tank_level - final_leg #Taking the fuel away to reach the final base
fuel_used+=final_leg #fuel used in reaching the base with no stop 
if tank_level<lower_tank_threshold:
    print('The rover did not make it to the base. The configuration does not work. The fuel was', -1*tank_level, 'too little.')
else:
    print('The rover made it! The configuration works. The rover made it to the base with', tank_level, 'of fuel left over. \n', fuel_used, 'of fuel has been used')




'''This block simulates the backwards journey of the rover.'''
print('\n \n \n Beginning the backwards journey. Fuel is at', tank_level)
fuel_used+=final_leg #Accounting for the fuel use to get back to the final outpost with no refill.

for backwards_journey in range(stops-2, -1, -1): 
    print('Leaving for stop', backwards_journey)
    tank_level=tank_level-stop_info[backwards_journey+1,1] #Removing fuel to reach the next outpost
    if tank_level<lower_tank_threshold:
        print(failed_statement, stop_info)
        break 
    print('\n Reached outpost', backwards_journey, 'with', tank_level, 'of fuel.')
    stop_info[backwards_journey, 2] = stop_info[backwards_journey,2]-stop_info[backwards_journey,1] #Removing one unit of fuel from the outpost to refuel the tank
    tank_level = tank_level+stop_info[backwards_journey,1]
    if tank_level>1.1:
        print(failed_statement, stop_info)
        break 
    print('The stops are at \n', stop_info, '\n Now have', tank_level, 'of fuel')

#Complete the final journey
forward_bit = stop_info[0,0] #The distance after the final stop
print('\n The journey left is', forward_bit , 'and the rover has', tank_level, 'of fuel')
tank_level=tank_level - forward_bit #Taking the fuel away to reach the final base
if tank_level<lower_tank_threshold:
    print('The rover did not make it to the LZ. The configuration does not work. The fuel was', -1*tank_level, 'too little.')
else:
    print('The rover made it! The configuration works. The rover made it to the original LZ with', tank_level, 'of fuel left over.\n', fuel_used, 'of fuel has been used.')

Establishing stop 0
Reached LZ with a tank of -1.3877787807814457e-17
[[0.01612903 0.01612903 0.96774194]
 [0.0327957  0.01666667 0.        ]
 [0.05003708 0.01724138 0.        ]
 [0.06789422 0.01785714 0.        ]
 [0.08641274 0.01851852 0.        ]
 [0.10564351 0.01923077 0.        ]
 [0.12564351 0.02       0.        ]
 [0.14647684 0.02083333 0.        ]
 [0.16821597 0.02173913 0.        ]
 [0.19094325 0.02272727 0.        ]
 [0.21475277 0.02380952 0.        ]
 [0.23975277 0.025      0.        ]
 [0.26606856 0.02631579 0.        ]
 [0.29384634 0.02777778 0.        ]
 [0.3232581  0.02941176 0.        ]
 [0.3545081  0.03125    0.        ]
 [0.38784143 0.03333333 0.        ]
 [0.42355572 0.03571429 0.        ]
 [0.46201726 0.03846154 0.        ]
 [0.50368393 0.04166667 0.        ]
 [0.54913847 0.04545455 0.        ]
 [0.59913847 0.05       0.        ]
 [0.65469403 0.05555556 0.        ]
 [0.71719403 0.0625     0.        ]
 [0.7886226  0.07142857 0.        ]
 [0.87195593 0.08333333 0.    